# Случайные события

In [1]:
import numpy as np
from itertools import permutations, chain

## Задача 1

10 карточек с написанными на них буквами А, А, И, И, О, С, С, Ц, Ц, Я перемешали и выложили в
ряд в случайном порядке. С какой вероятностью получится слово АССОЦИАЦИЯ?

### Способ 1

Обозначим вероятность вытащить 1ую букву 'А': $P(\text{А1})$

Тогда вероятность составить слово 'АССОЦИАЦИЯ': 

$$
P(\text{АССОЦИАЦИЯ}) = 
P(\text{А1})
P(\text{С1})
P(\text{С2})
P(\text{О1})
P(\text{Ц1})
P(\text{И1})
P(\text{А2})
P(\text{Ц2})
P(\text{И2})
P(\text{Я1}) = 
$$

$$
=
\frac{2}{10}\cdot\frac{2}{9}\cdot
\frac{1}{8}\cdot\frac{1}{7}\cdot
\frac{2}{6}\cdot\frac{2}{5}\cdot
\frac{1}{4}\cdot\frac{1}{3}\cdot\frac{1}{2}
=\frac{16}{10!}
$$

### Способ 2

Количество перестановок всех букв = 10!

Количество перестановок букв А, С, Ц, И = 2! (для каждой буквы)

Тогда:

$$
P(\text{АССОЦИАЦИЯ}) =  \frac{2!2!2!2!}{10!} = \frac{16}{10!}
$$

Проверим:

In [2]:
letters = ['А', 'А', 'И', 'И', 'О', 'С', 'С', 'Ц', 'Ц', 'Я']
word = ['А', 'С', 'С', 'О', 'Ц', 'И', 'А', 'Ц', 'И', 'Я']

letters = [ord(l) for l in letters]
word = [ord(l) for l in word]

# все возможные перестановки
perms = chain.from_iterable(permutations(letters))
perms = np.fromiter(perms, dtype=int).reshape(-1, len(letters))
# проверим совпадение всех символов
matches = np.all(perms == word, axis=1)
p = np.mean(matches)

In [3]:
p, 16 / np.math.factorial(10)

(4.409171075837742e-06, 4.409171075837742e-06)

## Задача 2

Какова вероятность того, что при подбрасывании 6 игральных костей хотя бы на двух костях
выпадет одинаковое число очков.

Найдем вероятность противоположного события: выпадут 6 разных чисел

Количество исходов, где 6 выпавших чисел разные: $6\cdot5\cdot4\cdot3\cdot2\cdot1$

Общее количество исходов: $6^6$

$$
P = 1 - \frac{6!}{6^6}
$$

Проверим:

In [4]:
outcomes = np.arange(1, 6+1)

n_dices = 6
n_trials = 10**6

results = np.random.choice(outcomes, n_dices*n_trials).reshape(n_trials, n_dices)
# отсортируем выпавшие числа
results = np.sort(results, axis=1)
# разность двух стоящих рядом чисел равна нулю => событие произошло
doubles = np.logical_not(np.diff(results))

p = np.mean(np.any(doubles, axis=1))

In [5]:
p, 1 - np.math.factorial(6) / 6**6

(0.984522, 0.9845679012345679)

## Задача 3

В ящике две партии по 100 резисторов: в первой партии — 10 бракованных, во второй — 20
бракованных. Из ящика извлечен 1 резистор.
Событие А = {извлеченный резистор — из первой партии}.
Событие B = {извлеченный резистор — бракованный}.
Зависимы ли события A и B? Ответ обосновать.

События зависимы, так как знание того, что одно из событий произошло, позволяет оценить вероятность возникновения другого:

- если резистор извлечен из первой партии, вероятность брака ниже среднего
- если резистор бракованный, то он скорее из второй партии, чем из первой

Проверим по определению независимых событий. Если события независимы, то:

$$
P(A \cap B) = P(A)P(B)
$$

где P(AB) - вероятность того, что резистор из первой партии бракованный

$$
P(A \cap B) = P(A|B)P(B) = P(B|A)P(A) = 
\frac{1}{3}\cdot\frac{30}{200} = 
\frac{10}{100}\cdot\frac{1}{2} = 
\frac{10}{200}
$$

$$
P(A)P(B) = \frac{1}{2} \cdot \frac{30}{200} = \frac{30}{400}
$$

$$
P(A \cap B) \neq P(A)P(B)
$$

Проверим:

In [6]:
n_trials = 10**6

bins = np.random.choice([1, 2], n_trials)
resistors = np.zeros(n_trials)

n_bin_1 = bins[bins == 1].shape[0]
n_bin_2 = bins[bins == 2].shape[0]

# 1 - резистор бракованный
resistors[bins == 1] = np.random.choice([0, 1], n_bin_1, p=[0.9, 0.1])
resistors[bins == 2] = np.random.choice([0, 1], n_bin_2, p=[0.8, 0.2])

p_ab = np.mean(np.logical_and(bins == 1, resistors == 1))
p_a = np.mean(bins == 1)
p_b = np.mean(resistors == 1)

In [7]:
p_ab, p_a * p_b

(0.049987, 0.074804925426)

## Задача 4

В микрорайоне девять машин технической службы. Для бесперебойной работы необходимо,
чтобы не меньше 8 машин были в исправном состоянии. Считая вероятность исправного
состояния для всех машин одинаковой и равной 0,9, найти вероятность бесперебойной работы
технической службы в микрорайоне.

$$
P = P(\text{8 машин из 9 в исправном состоянии}) + P(\text{9 машин в исправном состоянии})
$$

$$
P = {9\choose 8}P(\text{8 машин в исправном состоянии}) + P(\text{9 машин в исправном состоянии})
$$

$$
P = \frac{9!}{(9-8)!8!}\cdot 0.9^8\cdot0.1 + 0.9^9 = 2\cdot0.9^9
$$

Проверим:

In [8]:
n_trials = 10**6
n_machines = 9

# 1 - машина исправна
results = np.random.choice([0, 1], n_machines*n_trials, p=[0.1, 0.9]).reshape(n_trials, n_machines)
results = np.sum(results, axis=1)
# сумма >= 8 => бесперебойная работа
p = np.mean(results >= 8)

In [9]:
np.mean(results >= 8), 2*0.9**9

(0.77424, 0.7748409780000002)

## Задача 5

Имеется 10 винтовок, 4 из которых снабжены оптическим прицелом. Вероятность поражения
мишени стрелком равна: 0,95 — при выстреле из винтовки с оптическим прицелом, 0,8 — при
выстреле из винтовки без оптического прицела. Известно, что стрелок поразил мишень. Найдите
вероятность того, что он стрелял из винтовки без оптического прицела.

Пусть:  
- событие "o" - выбрана винтовка с оптическим прицелом,  
- событие "x" - выбрана винтовка без оптического прицела,  
- событие "y" - стрелок поразил мишень

Тогда:

$$
P(o) = 0.4, \enspace
P(x) = 0.6, \enspace
P(y|o) = 0.95, \enspace
P(y|x) = 0.8
$$

Нужно найти: $P(x|y)$

$$
P(x|y) = \frac{P(x \cap y)}{P(y)} = 
\frac{P(x)P(y|x)}{P(x)P(y|x)+P(o)P(y|o)}
$$

$$
P(x|y) =
\frac{0.6 \cdot 0.8}{0.6 \cdot 0.8 + 0.4 \cdot 0.95}
$$

Проверим:

In [10]:
n_trials = 10**6

# 0 - с оптическим прицелом, 1 - без оптического прицела
rifles = np.random.choice([0, 1], n_trials, p=[0.4, 0.6])
shots = np.zeros(n_trials)

n_rifles_0 = rifles[rifles == 0].shape[0]
n_rifles_1 = rifles[rifles == 1].shape[0]

# 0 - мимо, 1 - попадание
shots[rifles == 0] = np.random.choice([0, 1], n_rifles_0, p=[0.05, 0.95])
shots[rifles == 1] = np.random.choice([0, 1], n_rifles_1, p=[0.2, 0.8])
# индексы попаданий
hits = np.argwhere(shots == 1)

p = np.mean(rifles[hits] == 1)

In [11]:
p, 0.6*0.8 / (0.6*0.8 + 0.4*0.95)

(0.557492229522232, 0.5581395348837209)